In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataloaders import diagnosis_dataset
import numpy as np
from common import *

In [3]:
from trainer import Trainer, Evaluator
from models.Vanilla import ClassificationTrainer as BasicCT
from models.Hierarchical import ClassificationTrainer as HierCT
from Experiments.experiments import experiments, hierarchical_experiments, structured_experiments

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [4]:
data = diagnosis_dataset()

INFO - 2019-02-13 13:17:23,338 - Reading Structured data ...
INFO - 2019-02-13 13:17:23,518 - Reading Notes ...
INFO - 2019-02-13 13:17:37,910 - Stratifying ...


Basic Experiments
=================

In [ ]:
from PatientVec.models.baselines.LR import LR, LDA

In [ ]:
lr = LR({'vocab' : data.vocab, 'stop_words' : True, 'exp_name' : data.name, 'type' : 'multilabel'})
lr.train(train_data)
lr.evaluate(dev_data, save_results=True)
# lr.get_features(n=20)

In [ ]:
lda = LDA({'vocab' : data.vocab, 'stop_words' : True, 'exp_name' : data.name, 'type' : 'multilabel'})
lda.train(train_data)
lda.evaluate(dev_data, save_results=True)
# print(lda.get_topics(n=10))
# topics = lda.get_topics(n=10)
# print([topics[i] for i in np.argsort(lda.lda_classifier.coef_[0])])

In [ ]:
[topics[i] for i in np.argsort(lda.lda_classifier.coef_[0])]

In [ ]:
preds = outputs['predictions'][:, 0]

In [ ]:
from common import collapse_and_print_word_attn, print_sent_attn

In [ ]:
n = 50
collapse_and_print_word_attn(data.vocab, dev_data.X[n], outputs['word_attentions'][n])
print_sent_attn(data.vocab, dev_data.X[n], outputs['sentence_attentions'][n])

In [ ]:
outputs['sentence_attentions'][0]

In [ ]:
from scipy.stats import spearmanr, kendalltau

In [ ]:
corrs = [kendalltau(range(len(outputs['sentence_attentions'][i])), outputs['sentence_attentions'][i]) 
         for i in range(len(outputs['sentence_attentions']))]

In [ ]:
rho, leng = zip(*[(x[0], y) for x, y in zip(corrs, [len(z) for z in outputs['sentence_attentions']]) if x[0] == x[0]])

In [ ]:
plt.hist(rho, bins=30)

In [ ]:
pval, leng1 = zip(*[(x[1], y) for x, y in zip(corrs, [len(z) for z in outputs['sentence_attentions']]) if x[1] == x[1]])

Saving Models
==============

In [11]:
from common import generate_latex_tables
keys_to_use = ['macro_roc_auc', 'macro_pr_auc']
generate_latex_tables(data, keys_to_use)